In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [94]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [95]:
kc = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/kc_house_data.csv')

In [96]:
kc.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [97]:
kc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [98]:
# Transform date to a datetime data format
kc['date'] = pd.to_datetime(kc['date'])

In [99]:
# Extract hour, month, day into new columns
kc['month'] = kc['date'].dt.month
kc['day'] = kc['date'].dt.day
kc['year'] = kc['date'].dt.year

In [100]:
kc.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,day,year
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,...,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,...,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,6.574423,15.688197,2014.322954
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,...,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631,3.115308,8.635063,0.467616
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,...,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,1.000000,1.000000,2014.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,...,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000,4.000000,8.000000,2014.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,...,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000,6.000000,16.000000,2014.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,...,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000,9.000000,23.000000,2015.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,...,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,12.000000,31.000000,2015.000000


In [101]:
kc.drop_duplicates(inplace=True)

In [102]:
kc.drop('id', axis=1, inplace = True)
kc.drop('zipcode', axis=1, inplace = True)

In [103]:
kc.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,month,day,year
0,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,0,1955,0,47.5112,-122.257,1340,5650,10,13,2014
1,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,400,1951,1991,47.7210,-122.319,1690,7639,12,9,2014
2,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,...,0,1933,0,47.7379,-122.233,2720,8062,2,25,2015
3,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,...,910,1965,0,47.5208,-122.393,1360,5000,12,9,2014
4,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,0,1987,0,47.6168,-122.045,1800,7503,2,18,2015


In [104]:
# Create function to classify amounts
def total_cat(x):
    if x >0:
        return 1
    else:
        return 0

In [105]:
# Create new column by applying total_cat() function created above
kc['if_renovated'] = kc['yr_renovated'].apply(total_cat)
#bikes['count'] 是total_cost的x

In [106]:
correlations = kc.corr()['price']
# Filter the columns with correlation above 0.5
high_corr_columns = correlations[abs(correlations) >= 0.5].index.tolist()
print(f"Columns with correlation above or equal to 0.5 with 'x': {high_corr_columns}")


Columns with correlation above or equal to 0.5 with 'x': ['price', 'bathrooms', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15']


In [107]:
haha = ols("price ~ sqft_living15", data=kc).fit()
print(haha.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                 1.127e+04
Date:                Mon, 10 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:54:16   Log-Likelihood:            -3.0307e+05
No. Observations:               21613   AIC:                         6.061e+05
Df Residuals:                   21611   BIC:                         6.062e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -8.281e+04   6208.029    -13.339

In [108]:
results = ols("price ~ bathrooms+sqft_living+grade+sqft_above", data=kc).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     6433.
Date:                Mon, 10 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:54:16   Log-Likelihood:            -2.9913e+05
No. Observations:               21613   AIC:                         5.983e+05
Df Residuals:                   21608   BIC:                         5.983e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -6.502e+05   1.35e+04    -48.132      

In [109]:
kc['log_sqft_living'] = np.log(kc['sqft_living'])
kc['log_grade'] = np.log(kc['grade'])
kc['log_price'] = np.log(kc['price'])
kc['log_sqft_above'] = np.log(kc['sqft_above'])

In [110]:
results_log = ols("log_price ~ bathrooms + log_sqft_living + log_grade+ log_sqft_above", data=kc).fit()
print(results_log.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     6399.
Date:                Mon, 10 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:54:16   Log-Likelihood:                -8365.0
No. Observations:               21613   AIC:                         1.674e+04
Df Residuals:                   21608   BIC:                         1.678e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.8135      0.061    1

In [111]:
kc.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_living15,sqft_lot15,month,day,year,if_renovated,log_sqft_living,log_grade,log_price,log_sqft_above
0,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,1340,5650,10,13,2014,0,7.073270,1.945910,12.309982,7.073270
1,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,1690,7639,12,9,2014,1,7.851661,1.945910,13.195614,7.682482
2,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,...,2720,8062,2,25,2015,0,6.646391,1.791759,12.100712,6.646391
3,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,...,1360,5000,12,9,2014,0,7.580700,1.945910,13.311329,6.956545
4,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,1800,7503,2,18,2015,0,7.426549,2.079442,13.142166,7.426549


In [115]:
6.8135 + -0.0026*3 + 0.5908*7.073270 + 1.6110*1.945910- 0.2004*7.073270
#first line of data

12.701965618000001

In [117]:
pred_vals = results_log.predict(kc[['bathrooms','log_sqft_living','log_grade','log_sqft_above']])

In [118]:
pred_vals

0        12.707179
1        13.041690
2        12.292185
3        13.025119
4        13.057600
           ...    
21608    13.019776
21609    13.180613
21610    12.650949
21611    13.037241
21612    12.650949
Length: 21613, dtype: float64

In [120]:
res = pd.concat([pred_vals.to_frame().rename(columns={0:'y_hat'}),kc['price']], axis=1)

In [121]:
res

,y_hat,price
0,12.707179,221900.0
1,13.041690,538000.0
2,12.292185,180000.0
3,13.025119,604000.0
4,13.057600,510000.0
...,...,...
21608,13.019776,360000.0
21609,13.180613,400000.0
21610,12.650949,402101.0
21611,13.037241,400000.0


In [128]:
res['log_price'] = kc['log_price']

In [129]:
res['error'] = res['y_hat'] - res['log_price']

In [130]:
res.head(1)

,y_hat,price,error,sq_error,log_price
0,12.707179,221900.0,0.397197,4.923397e+10,12.309982


In [131]:
res['sq_error'] = res['error']**2

In [132]:
np.sqrt(res['sq_error'].mean())

0.35632826402534085